<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/retrieval/Retrievals_approaches_evaluation_phi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 14.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.76-cp310-cp310-linux_x86_64.whl size=80964247 sha256=786cfbd21c667c94c081d117736d108918f9780d3601bcf3b91e5617824680ad
  Stored in directory: /root/.cache/pip/wheels/a0/e5/04/a5fa9e60033548f205f0db5f6ab6f59cd27bd0da7f9c51cfe7
Successfully built llama-cpp-python


In [2]:
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-gguf Phi-3-mini-4k-instruct-fp16.gguf --local-dir ./models --local-dir-use-symlinks False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Phi-3-mini-4k-instruct-fp16.gguf: 100% 7.64G/7.64G [00:53<00:00, 143MB/s]
Download complete. Moving file to models/Phi-3-mini-4k-instruct-fp16.gguf
models/Phi-3-mini-4k-instruct-fp16.gguf


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install langchain langchain-core langchain-community --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.7 MB/s eta 0:00:00


In [5]:
from langchain_community.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain_core.output_parsers import JsonOutputParser
from json import JSONDecodeError
from langchain_core.exceptions import OutputParserException
import pandas as pd
import os

In [6]:
TEMP = 0
N_CTX = 4096
N_GPU_L = -1

llm = LlamaCpp(
    model_path="/content/models/Phi-3-mini-4k-instruct-fp16.gguf",
    temperature=TEMP,
    n_ctx=N_CTX,
    n_gpu_layers = N_GPU_L,
    verbose=True
)

llama_model_loader: loaded meta data with 23 key-value pairs and 195 tensors from /content/models/Phi-3-mini-4k-instruct-fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.head_count

In [7]:
### Retrieval Grader for approach 1

prompt_1 = PromptTemplate(
    template="""
    <|assistant|> You are a grader assessing relevance of a retrieved document to a user question. If the document contains keywords related to the user question, grade it as relevant.
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination. <|end|>
    <|user|> Here is the retrieved document: \n\n {document} \n\n Here is the user question: {question} <|end|>
    <|assistant|>
    """,
    input_variables=["question", "document"],
)

retrieval_grader_1 = prompt_1 | llm | JsonOutputParser()

In [8]:
### Retrieval Grader for approach 2

prompt_2 = PromptTemplate(
    template="""
    <|assistant|> You are a grader assessing relevance of a retrieved document to a user question. If the document contains keywords related to the user question, grade it as relevant.
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination. <|end|>
    <|user|> Here is the retrieved document: \n\n {document} \n\n Here is the user question: {question_name} <|end|>
    <|assistant|>
    """,
    input_variables=["question_name", "document"],
)

retrieval_grader_2 = prompt_2 | llm | JsonOutputParser()

In [9]:
folder_path = '/content/drive/MyDrive/Thesis/retrievals/'

dataframes = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('.json'):
        approach_name = file_name
        dataframes.append(approach_name)

In [10]:
def calculate_results(approach_number:int, distance: str):
  results_list = []

  for i in range(len(dataframes)):

    if approach_number == 1:
      approach = pd.read_json(os.path.join(folder_path + dataframes[i]))
      try:
        for j in range(len(approach)):
          question = approach['question'][j]
          doc_txt = approach[distance][j]['page_content']
          try:
            answer = retrieval_grader_1.invoke({"question": question, "document": doc_txt})
            results_list.append(pd.DataFrame({"approach": 1,"distance": distance, "question": question, "answer": [answer], "document": doc_txt}))
          except JSONDecodeError as e:
            print(f"JSONDecodeError occurred for question: {question}. Skipping...")
            continue
          except OutputParserException as e:
            print(f"OutputParserException occurred for question: {question}. Skipping...")
            continue
      except KeyError as e:
        print(f"KeyError occurred: {e}. Skipping approach {approach_number} for dataframe {dataframes[i]}")
        continue


    elif approach_number == 2:
      approach = pd.read_json(os.path.join(folder_path + dataframes[i]))
      try:
        for j in range(len(approach)):
          question = approach['question_name'][j]
          doc_txt = approach[distance][j]['page_content']
          try:
            answer = retrieval_grader_2.invoke({"question_name": question, "document": doc_txt})
            results_list.append(pd.DataFrame({"approach": 2,"distance": distance, "question": question, "answer": [answer], "document": doc_txt}))
          except JSONDecodeError as e:
            print(f"JSONDecodeError occurred for question: {question}. Skipping...")
            continue
          except OutputParserException as e:
            print(f"OutputParserException occurred for question: {question}. Skipping...")
            continue
      except KeyError as e:
        print(f"KeyError occurred: {e}. Skipping approach {approach_number} for dataframe {dataframes[i]}")
        continue

  if results_list:
    results = pd.concat(results_list, ignore_index=True)
    score = 0
    for i in range(len(results)):
      if (results['answer'][i] is not None) and (results['answer'][i]['score'] == 'yes'):
        score += 1
    total_score = score/len(results)
    print(f'Total score for approach {approach_number} and distance function {distance} is {total_score}')
    results.to_json(f'/content/drive/MyDrive/Thesis/evaluation/eval_{approach_number}_{distance}.json')
  else:
    print(f"No results to concatenate for approach {approach_number} and distance {distance}")

In [ ]:
calculate_results(1, 'cosine')


llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =       3.72 ms /     7 runs   (    0.53 ms per token,  1879.70 tokens per second)
llama_print_timings: prompt eval time =    4585.43 ms /   786 tokens (    5.83 ms per token,   171.41 tokens per second)
llama_print_timings:        eval time =     273.68 ms /     6 runs   (   45.61 ms per token,    21.92 tokens per second)
llama_print_timings:       total time =    4966.56 ms /   792 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =      42.14 ms /    68 runs   (    0.62 ms per token,  1613.55 tokens per second)
llama_print_timings: prompt eval time =    3136.35 ms /   608 tokens (    5.16 ms per token,   193.86 tokens per second)
llama_print_timings:        eval time =    2611.39 ms /    68 runs   (   38.40 ms per token,    26.04 tokens per second)
llama_print_timings:       total time =    5876.41 ms /   676 

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_cosine_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_ip_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_l2_2.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =      40.66 ms /    65 runs   (    0.63 ms per token,  1598.70 tokens per second)
llama_print_timings: prompt eval time =    2776.16 ms /   516 tokens (    5.38 ms per token,   185.87 tokens per second)
llama_print_timings:        eval time =    2541.68 ms /    64 runs   (   39.71 ms per token,    25.18 tokens per second)
llama_print_timings:       total time =    5424.15 ms /   580 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =      47.64 ms /    80 runs   (    0.60 ms per token,  1679.37 tokens per second)
llama_print_timings: prompt eval time =    3559.55 ms /   655 tokens (    5.43 ms per token,   184.01 tokens per second)
llama_print_timings:        eval time =    3162.33 ms /    79 runs   (   40.03 ms per token,    24.98 tokens per second)
llama_print_timings:       to

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_cosine_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_ip_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_l2_2.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =      41.13 ms /    70 runs   (    0.59 ms per token,  1702.04 tokens per second)
llama_print_timings: prompt eval time =    2976.92 ms /   551 tokens (    5.40 ms per token,   185.09 tokens per second)
llama_print_timings:        eval time =    2736.69 ms /    69 runs   (   39.66 ms per token,    25.21 tokens per second)
llama_print_timings:       total time =    5823.78 ms /   620 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =      45.22 ms /    62 runs   (    0.73 ms per token,  1371.20 tokens per second)
llama_print_timings: prompt eval time =    2931.07 ms /   547 tokens (    5.36 ms per token,   186.62 tokens per second)
llama_print_timings:        eval time =    2442.94 ms /    61 runs   (   40.05 ms per token,    24.97 tokens per second)
llama_print_timings:       to

...

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_MICROSOFT CORP_cosine_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_MICROSOFT CORP_ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_MICROSOFT CORP_ip_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_MICROSOFT CORP_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_MICROSOFT CORP_l2_2.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =      57.17 ms /    98 runs   (    0.58 ms per token,  1714.25 tokens per second)
llama_print_timings: prompt eval time =    2607.90 ms /   486 tokens (    5.37 ms per token,   186.36 tokens per second)
llama_print_timings:        eval time =    3826.57 ms /    97 runs   (   39.45 ms per token,    25.35 tokens per second)
llama_print_timings:       total time =    6590.66 ms /   583 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =       3.38 ms /     7 runs   (    0.48 ms per token,  2071.62 tokens per second)
llama_print_timings: prompt eval time =    2937.67 ms /   549 tokens (    5.35 ms per token,   186.88 tokens per second)
llama_print_timings:        eval time =     239.67 ms /     6 runs   (   39.94 ms per token,    25.03 tokens per second)
llama_print_timings:       to

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._cosine_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._ip_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._l2_2.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =       3.75 ms /     7 runs   (    0.54 ms per token,  1868.66 tokens per second)
llama_print_timings: prompt eval time =    2353.15 ms /   435 tokens (    5.41 ms per token,   184.86 tokens per second)
llama_print_timings:        eval time =     233.37 ms /     6 runs   (   38.90 ms per token,    25.71 tokens per second)
llama_print_timings:       total time =    2600.27 ms /   441 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =       6.28 ms /    12 runs   (    0.52 ms per token,  1912.05 tokens per second)
llama_print_timings: prompt eval time =    2881.73 ms /   544 tokens (    5.30 ms per token,   188.78 tokens per second)
llama_print_timings:        eval time =     434.79 ms /    11 runs   (   39.53 ms per token,    25.30 tokens per second)
llama_print_timings:       to

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AES CORP_cosine_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_AES CORP_ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AES CORP_ip_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_AES CORP_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AES CORP_l2_2.json
Total score for approach 1 and distance function cosine is 0.5715990453460621


In [11]:
calculate_results(1, 'ip')

KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_cosine_2.json



llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      48.12 ms /    88 runs   (    0.55 ms per token,  1828.84 tokens per second)
llama_print_timings: prompt eval time =    3729.57 ms /   663 tokens (    5.63 ms per token,   177.77 tokens per second)
llama_print_timings:        eval time =    3303.89 ms /    87 runs   (   37.98 ms per token,    26.33 tokens per second)
llama_print_timings:       total time =    7217.84 ms /   750 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      67.36 ms /   102 runs   (    0.66 ms per token,  1514.34 tokens per second)
llama_print_timings: prompt eval time =    1435.49 ms /   276 tokens (    5.20 ms per token,   192.27 tokens per second)
llama_print_timings:        eval time =    3835.24 ms /   101 runs   (   37.97 ms per token,    26.33 tokens per second)
llama_print_timings:       total time =    5447.27 ms /   377 

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_ip_2.json
KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_l2_2.json
KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_cosine_2.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      35.11 ms /    65 runs   (    0.54 ms per token,  1851.48 tokens per second)
llama_print_timings: prompt eval time =    2767.39 ms /   516 tokens (    5.36 ms per token,   186.46 tokens per second)
llama_print_timings:        eval time =    2509.76 ms /    64 runs   (   39.22 ms per token,    25.50 tokens per second)
llama_print_timings:       total time =    5365.12 ms /   580 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      54.76 ms /    80 runs   (    0.68 ms per token,  1461.00 tokens per second)
llama_print_timings: prompt eval time =    3469.26 ms /   655 tokens (    5.30 ms per token,   188.80 tokens per second)
llama_print_timings:        eval time =    3140.31 ms /    79 runs   (   39.75 ms per token,    25.16 tokens per second)
llama_print_timings:       to

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_ip_2.json
KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_l2_2.json
KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_PayPal Holdings, Inc._cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_PayPal Holdings, Inc._cosine_2.json


...

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_MICROSOFT CORP_ip_2.json
KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_MICROSOFT CORP_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_MICROSOFT CORP_l2_2.json
KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._cosine_2.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      52.80 ms /    98 runs   (    0.54 ms per token,  1856.03 tokens per second)
llama_print_timings: prompt eval time =    2619.93 ms /   486 tokens (    5.39 ms per token,   185.50 tokens per second)
llama_print_timings:        eval time =    3804.98 ms /    97 runs   (   39.23 ms per token,    25.49 tokens per second)
llama_print_timings:       total time =    6559.80 ms /   583 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      44.74 ms /    84 runs   (    0.53 ms per token,  1877.47 tokens per second)
llama_print_timings: prompt eval time =    2583.40 ms /   493 tokens (    5.24 ms per token,   190.83 tokens per second)
llama_print_timings:        eval time =    3259.29 ms /    83 runs   (   39.27 ms per token,    25.47 tokens per second)
llama_print_timings:       to

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._ip_2.json
KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._l2_2.json
KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_AES CORP_cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AES CORP_cosine_2.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =       3.43 ms /     7 runs   (    0.49 ms per token,  2039.03 tokens per second)
llama_print_timings: prompt eval time =    2282.72 ms /   435 tokens (    5.25 ms per token,   190.56 tokens per second)
llama_print_timings:        eval time =     226.14 ms /     6 runs   (   37.69 ms per token,    26.53 tokens per second)
llama_print_timings:       total time =    2523.53 ms /   441 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      41.37 ms /    80 runs   (    0.52 ms per token,  1933.67 tokens per second)
llama_print_timings: prompt eval time =    3232.25 ms /   624 tokens (    5.18 ms per token,   193.05 tokens per second)
llama_print_timings:        eval time =    3090.04 ms /    80 runs   (   38.63 ms per token,    25.89 tokens per second)
llama_print_timings:       to

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AES CORP_ip_2.json
KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_AES CORP_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AES CORP_l2_2.json
Total score for approach 1 and distance function ip is 0.5670588235294117


In [12]:
calculate_results(1, 'l2')

KeyError occurred: 'l2'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_cosine_2.json
KeyError occurred: 'l2'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_ip_2.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =       3.26 ms /     7 runs   (    0.47 ms per token,  2146.58 tokens per second)
llama_print_timings: prompt eval time =    3474.01 ms /   655 tokens (    5.30 ms per token,   188.54 tokens per second)
llama_print_timings:        eval time =     235.53 ms /     6 runs   (   39.25 ms per token,    25.48 tokens per second)
llama_print_timings:       total time =    3725.16 ms /   661 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      36.98 ms /    68 runs   (    0.54 ms per token,  1839.08 tokens per second)
llama_print_timings: prompt eval time =    3204.68 ms /   608 tokens (    5.27 ms per token,   189.72 tokens per second)
llama_print_timings:        eval time =    2676.93 ms /    68 runs   (   39.37 ms per token,    25.40 tokens per second)
llama_print_timings:       to

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_l2_2.json
KeyError occurred: 'l2'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_cosine_2.json
KeyError occurred: 'l2'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_ip_2.json



llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      35.43 ms /    65 runs   (    0.55 ms per token,  1834.76 tokens per second)
llama_print_timings: prompt eval time =    2719.10 ms /   516 tokens (    5.27 ms per token,   189.77 tokens per second)
llama_print_timings:        eval time =    2516.46 ms /    64 runs   (   39.32 ms per token,    25.43 tokens per second)
llama_print_timings:       total time =    5322.10 ms /   580 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      54.39 ms /    80 runs   (    0.68 ms per token,  1470.83 tokens per second)
llama_print_timings: prompt eval time =    3472.69 ms /   655 tokens (    5.30 ms per token,   188.61 tokens per second)
llama_print_timings:        eval time =    3136.77 ms /    79 runs   (   39.71 ms per token,    25.19 tokens per second)
llama_print_timings:       total time =    6741.85 ms /   734 

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_l2_2.json
KeyError occurred: 'l2'. Skipping approach 1 for dataframe results_approach_PayPal Holdings, Inc._cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_PayPal Holdings, Inc._cosine_2.json
KeyError occurred: 'l2'. Skipping approach 1 for dataframe results_approach_PayPal Holdings, Inc._ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_PayPal Holdings, Inc._ip_2.json


...

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_MICROSOFT CORP_l2_2.json
KeyError occurred: 'l2'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._cosine_2.json
KeyError occurred: 'l2'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._ip_2.json



llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =       3.36 ms /     7 runs   (    0.48 ms per token,  2082.09 tokens per second)
llama_print_timings: prompt eval time =    2895.78 ms /   549 tokens (    5.27 ms per token,   189.59 tokens per second)
llama_print_timings:        eval time =     236.07 ms /     6 runs   (   39.34 ms per token,    25.42 tokens per second)
llama_print_timings:       total time =    3147.60 ms /   555 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      66.29 ms /    98 runs   (    0.68 ms per token,  1478.33 tokens per second)
llama_print_timings: prompt eval time =    2548.63 ms /   486 tokens (    5.24 ms per token,   190.69 tokens per second)
llama_print_timings:        eval time =    3807.47 ms /    97 runs   (   39.25 ms per token,    25.48 tokens per second)
llama_print_timings:       total time =    6525.54 ms /   583 

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._l2_2.json
KeyError occurred: 'l2'. Skipping approach 1 for dataframe results_approach_AES CORP_cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AES CORP_cosine_2.json
KeyError occurred: 'l2'. Skipping approach 1 for dataframe results_approach_AES CORP_ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AES CORP_ip_2.json



llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =       6.01 ms /    12 runs   (    0.50 ms per token,  1997.67 tokens per second)
llama_print_timings: prompt eval time =    2863.09 ms /   544 tokens (    5.26 ms per token,   190.00 tokens per second)
llama_print_timings:        eval time =     431.27 ms /    11 runs   (   39.21 ms per token,    25.51 tokens per second)
llama_print_timings:       total time =    3316.07 ms /   555 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =       4.26 ms /     7 runs   (    0.61 ms per token,  1641.65 tokens per second)
llama_print_timings: prompt eval time =    2308.11 ms /   435 tokens (    5.31 ms per token,   188.47 tokens per second)
llama_print_timings:        eval time =     234.85 ms /     6 runs   (   39.14 ms per token,    25.55 tokens per second)
llama_print_timings:       total time =    2559.17 ms /   441 

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AES CORP_l2_2.json
Total score for approach 1 and distance function l2 is 0.5722819593787336


In [13]:
calculate_results(2, 'cosine')

Llama.generate: prefix-match hit


KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_cosine_1.json



llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      69.31 ms /   109 runs   (    0.64 ms per token,  1572.55 tokens per second)
llama_print_timings: prompt eval time =    5940.10 ms /  1079 tokens (    5.51 ms per token,   181.65 tokens per second)
llama_print_timings:        eval time =    4438.85 ms /   108 runs   (   41.10 ms per token,    24.33 tokens per second)
llama_print_timings:       total time =   10574.71 ms /  1187 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      46.33 ms /    86 runs   (    0.54 ms per token,  1856.17 tokens per second)
llama_print_timings: prompt eval time =    5028.75 ms /   922 tokens (    5.45 ms per token,   183.35 tokens per second)
llama_print_timings:        eval time =    3421.88 ms /    85 runs   (   40.26 ms per token,    24.84 tokens per second)
llama_print_timings:       total time =    8574.26 ms /  1007 

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_ip_1.json
KeyError occurred: 'cosine'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_ip_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_l2_1.json
KeyError occurred: 'cosine'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_l2_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AMAZON COM INC_cosine_1.json



llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      64.37 ms /   116 runs   (    0.55 ms per token,  1802.00 tokens per second)
llama_print_timings: prompt eval time =    5281.77 ms /   965 tokens (    5.47 ms per token,   182.70 tokens per second)
llama_print_timings:        eval time =    4645.39 ms /   115 runs   (   40.39 ms per token,    24.76 tokens per second)
llama_print_timings:       total time =   10096.61 ms /  1080 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      49.57 ms /    76 runs   (    0.65 ms per token,  1533.22 tokens per second)
llama_print_timings: prompt eval time =    2826.41 ms /   532 tokens (    5.31 ms per token,   188.22 tokens per second)
llama_print_timings:        eval time =    2949.97 ms /    75 runs   (   39.33 ms per token,    25.42 tokens per second)
llama_print_timings:       total time =    5902.50 ms /   607 

...

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_MICROSOFT CORP_ip_1.json
KeyError occurred: 'cosine'. Skipping approach 2 for dataframe results_approach_MICROSOFT CORP_ip_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_MICROSOFT CORP_l2_1.json
KeyError occurred: 'cosine'. Skipping approach 2 for dataframe results_approach_MICROSOFT CORP_l2_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Ulta Beauty, Inc._cosine_1.json



llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      19.05 ms /    37 runs   (    0.51 ms per token,  1942.36 tokens per second)
llama_print_timings: prompt eval time =    6048.71 ms /  1094 tokens (    5.53 ms per token,   180.86 tokens per second)
llama_print_timings:        eval time =    1466.05 ms /    36 runs   (   40.72 ms per token,    24.56 tokens per second)
llama_print_timings:       total time =    7578.93 ms /  1130 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      64.49 ms /    94 runs   (    0.69 ms per token,  1457.50 tokens per second)
llama_print_timings: prompt eval time =    3024.93 ms /   570 tokens (    5.31 ms per token,   188.43 tokens per second)
llama_print_timings:        eval time =    3661.31 ms /    93 runs   (   39.37 ms per token,    25.40 tokens per second)
llama_print_timings:       total time =    6854.98 ms /   663 

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Ulta Beauty, Inc._ip_1.json
KeyError occurred: 'cosine'. Skipping approach 2 for dataframe results_approach_Ulta Beauty, Inc._ip_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Ulta Beauty, Inc._l2_1.json
KeyError occurred: 'cosine'. Skipping approach 2 for dataframe results_approach_Ulta Beauty, Inc._l2_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AES CORP_cosine_1.json



llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =     113.11 ms /   220 runs   (    0.51 ms per token,  1945.04 tokens per second)
llama_print_timings: prompt eval time =    4512.77 ms /   831 tokens (    5.43 ms per token,   184.14 tokens per second)
llama_print_timings:        eval time =    8785.45 ms /   219 runs   (   40.12 ms per token,    24.93 tokens per second)
llama_print_timings:       total time =   13598.95 ms /  1050 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      37.68 ms /    71 runs   (    0.53 ms per token,  1884.24 tokens per second)
llama_print_timings: prompt eval time =    3108.31 ms /   581 tokens (    5.35 ms per token,   186.92 tokens per second)
llama_print_timings:        eval time =    2757.10 ms /    70 runs   (   39.39 ms per token,    25.39 tokens per second)
llama_print_timings:       total time =    5961.08 ms /   651 

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AES CORP_ip_1.json
KeyError occurred: 'cosine'. Skipping approach 2 for dataframe results_approach_AES CORP_ip_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AES CORP_l2_1.json
KeyError occurred: 'cosine'. Skipping approach 2 for dataframe results_approach_AES CORP_l2_2.json
Total score for approach 2 and distance function cosine is 0.3793939393939394


In [14]:
calculate_results(2, 'ip')

Llama.generate: prefix-match hit


KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_cosine_1.json
KeyError occurred: 'ip'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_cosine_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_ip_1.json



llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      43.93 ms /    81 runs   (    0.54 ms per token,  1843.67 tokens per second)
llama_print_timings: prompt eval time =    5224.77 ms /   947 tokens (    5.52 ms per token,   181.25 tokens per second)
llama_print_timings:        eval time =    3229.38 ms /    80 runs   (   40.37 ms per token,    24.77 tokens per second)
llama_print_timings:       total time =    8579.29 ms /  1027 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      57.84 ms /    85 runs   (    0.68 ms per token,  1469.62 tokens per second)
llama_print_timings: prompt eval time =    4340.14 ms /   806 tokens (    5.38 ms per token,   185.71 tokens per second)
llama_print_timings:        eval time =    3373.15 ms /    84 runs   (   40.16 ms per token,    24.90 tokens per second)
llama_print_timings:       total time =    7864.26 ms /   890 

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_l2_1.json
KeyError occurred: 'ip'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_l2_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AMAZON COM INC_cosine_1.json
KeyError occurred: 'ip'. Skipping approach 2 for dataframe results_approach_AMAZON COM INC_cosine_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AMAZON COM INC_ip_1.json



llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      49.61 ms /    76 runs   (    0.65 ms per token,  1531.83 tokens per second)
llama_print_timings: prompt eval time =    2832.12 ms /   532 tokens (    5.32 ms per token,   187.85 tokens per second)
llama_print_timings:        eval time =    2948.91 ms /    75 runs   (   39.32 ms per token,    25.43 tokens per second)
llama_print_timings:       total time =    5906.49 ms /   607 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      59.35 ms /   108 runs   (    0.55 ms per token,  1819.65 tokens per second)
llama_print_timings: prompt eval time =    2760.18 ms /   522 tokens (    5.29 ms per token,   189.12 tokens per second)
llama_print_timings:        eval time =    4185.88 ms /   107 runs   (   39.12 ms per token,    25.56 tokens per second)
llama_print_timings:       total time =    7096.37 ms /   629 

...

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_MICROSOFT CORP_l2_1.json
KeyError occurred: 'ip'. Skipping approach 2 for dataframe results_approach_MICROSOFT CORP_l2_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Ulta Beauty, Inc._cosine_1.json
KeyError occurred: 'ip'. Skipping approach 2 for dataframe results_approach_Ulta Beauty, Inc._cosine_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Ulta Beauty, Inc._ip_1.json



llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      38.56 ms /    70 runs   (    0.55 ms per token,  1815.35 tokens per second)
llama_print_timings: prompt eval time =    6182.61 ms /  1120 tokens (    5.52 ms per token,   181.15 tokens per second)
llama_print_timings:        eval time =    2820.06 ms /    69 runs   (   40.87 ms per token,    24.47 tokens per second)
llama_print_timings:       total time =    9110.71 ms /  1189 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      48.45 ms /    89 runs   (    0.54 ms per token,  1837.13 tokens per second)
llama_print_timings: prompt eval time =    6496.69 ms /  1168 tokens (    5.56 ms per token,   179.78 tokens per second)
llama_print_timings:        eval time =    3616.97 ms /    88 runs   (   41.10 ms per token,    24.33 tokens per second)
llama_print_timings:       total time =   10247.68 ms /  1256 

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Ulta Beauty, Inc._l2_1.json
KeyError occurred: 'ip'. Skipping approach 2 for dataframe results_approach_Ulta Beauty, Inc._l2_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AES CORP_cosine_1.json
KeyError occurred: 'ip'. Skipping approach 2 for dataframe results_approach_AES CORP_cosine_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AES CORP_ip_1.json



llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      41.38 ms /    77 runs   (    0.54 ms per token,  1860.80 tokens per second)
llama_print_timings: prompt eval time =    4380.60 ms /   810 tokens (    5.41 ms per token,   184.91 tokens per second)
llama_print_timings:        eval time =    3026.86 ms /    76 runs   (   39.83 ms per token,    25.11 tokens per second)
llama_print_timings:       total time =    7517.73 ms /   886 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      43.43 ms /    71 runs   (    0.61 ms per token,  1634.74 tokens per second)
llama_print_timings: prompt eval time =    3104.11 ms /   581 tokens (    5.34 ms per token,   187.17 tokens per second)
llama_print_timings:        eval time =    2758.81 ms /    70 runs   (   39.41 ms per token,    25.37 tokens per second)
llama_print_timings:       total time =    5974.65 ms /   651 

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AES CORP_l2_1.json
KeyError occurred: 'ip'. Skipping approach 2 for dataframe results_approach_AES CORP_l2_2.json
Total score for approach 2 and distance function ip is 0.35714285714285715


In [15]:
calculate_results(2, 'l2')

Llama.generate: prefix-match hit


KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_cosine_1.json
KeyError occurred: 'l2'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_cosine_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_ip_1.json
KeyError occurred: 'l2'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_ip_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_l2_1.json



llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      65.72 ms /   109 runs   (    0.60 ms per token,  1658.55 tokens per second)
llama_print_timings: prompt eval time =    5956.29 ms /  1077 tokens (    5.53 ms per token,   180.82 tokens per second)
llama_print_timings:        eval time =    4423.15 ms /   108 runs   (   40.96 ms per token,    24.42 tokens per second)
llama_print_timings:       total time =   10568.34 ms /  1185 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      52.69 ms /    86 runs   (    0.61 ms per token,  1632.10 tokens per second)
llama_print_timings: prompt eval time =    5029.92 ms /   922 tokens (    5.46 ms per token,   183.30 tokens per second)
llama_print_timings:        eval time =    3440.90 ms /    85 runs   (   40.48 ms per token,    24.70 tokens per second)
llama_print_timings:       total time =    8618.36 ms /  1007 

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AMAZON COM INC_cosine_1.json
KeyError occurred: 'l2'. Skipping approach 2 for dataframe results_approach_AMAZON COM INC_cosine_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AMAZON COM INC_ip_1.json
KeyError occurred: 'l2'. Skipping approach 2 for dataframe results_approach_AMAZON COM INC_ip_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AMAZON COM INC_l2_1.json



llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      61.35 ms /   116 runs   (    0.53 ms per token,  1890.91 tokens per second)
llama_print_timings: prompt eval time =    5271.48 ms /   965 tokens (    5.46 ms per token,   183.06 tokens per second)
llama_print_timings:        eval time =    4642.26 ms /   115 runs   (   40.37 ms per token,    24.77 tokens per second)
llama_print_timings:       total time =   10067.84 ms /  1080 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      50.94 ms /    76 runs   (    0.67 ms per token,  1492.07 tokens per second)
llama_print_timings: prompt eval time =    2815.97 ms /   532 tokens (    5.29 ms per token,   188.92 tokens per second)
llama_print_timings:        eval time =    2952.53 ms /    75 runs   (   39.37 ms per token,    25.40 tokens per second)
llama_print_timings:       total time =    5888.38 ms /   607 

...

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Ulta Beauty, Inc._cosine_1.json
KeyError occurred: 'l2'. Skipping approach 2 for dataframe results_approach_Ulta Beauty, Inc._cosine_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Ulta Beauty, Inc._ip_1.json
KeyError occurred: 'l2'. Skipping approach 2 for dataframe results_approach_Ulta Beauty, Inc._ip_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Ulta Beauty, Inc._l2_1.json



llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      23.75 ms /    37 runs   (    0.64 ms per token,  1557.89 tokens per second)
llama_print_timings: prompt eval time =    6036.59 ms /  1094 tokens (    5.52 ms per token,   181.23 tokens per second)
llama_print_timings:        eval time =    1489.70 ms /    36 runs   (   41.38 ms per token,    24.17 tokens per second)
llama_print_timings:       total time =    7600.85 ms /  1130 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      59.96 ms /    94 runs   (    0.64 ms per token,  1567.79 tokens per second)
llama_print_timings: prompt eval time =    3071.80 ms /   570 tokens (    5.39 ms per token,   185.56 tokens per second)
llama_print_timings:        eval time =    3652.44 ms /    93 runs   (   39.27 ms per token,    25.46 tokens per second)
llama_print_timings:       total time =    6870.86 ms /   663 

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AES CORP_cosine_1.json
KeyError occurred: 'l2'. Skipping approach 2 for dataframe results_approach_AES CORP_cosine_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AES CORP_ip_1.json
KeyError occurred: 'l2'. Skipping approach 2 for dataframe results_approach_AES CORP_ip_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AES CORP_l2_1.json



llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =     126.48 ms /   220 runs   (    0.57 ms per token,  1739.45 tokens per second)
llama_print_timings: prompt eval time =    4477.00 ms /   831 tokens (    5.39 ms per token,   185.62 tokens per second)
llama_print_timings:        eval time =    8824.29 ms /   219 runs   (   40.29 ms per token,    24.82 tokens per second)
llama_print_timings:       total time =   13661.29 ms /  1050 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     424.96 ms
llama_print_timings:      sample time =      38.42 ms /    71 runs   (    0.54 ms per token,  1847.90 tokens per second)
llama_print_timings: prompt eval time =    3073.12 ms /   581 tokens (    5.29 ms per token,   189.06 tokens per second)
llama_print_timings:        eval time =    2751.91 ms /    70 runs   (   39.31 ms per token,    25.44 tokens per second)
llama_print_timings:       total time =    5923.13 ms /   651 

Total score for approach 2 and distance function l2 is 0.3688029020556227
